In [15]:
import cv2
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import os

In [16]:
type = input("Enter image type (dark/bright):")

In [22]:
if(type == "dark"):
    dark =1
    current_dir = os.getcwd()  # Your PyCharm project folder
    input_dir = current_dir

    for filename in os.listdir(input_dir):
        if not (filename.endswith(".jpg") or filename.endswith(".png")):
            continue  # Skip non-images

        img_path = os.path.join(input_dir, filename)
        img = cv2.imread(img_path)
        if img is None:
            print(f"❌ Failed to load: {filename}")
            continue

        if len(img.shape) == 2:
            avg_brightness = np.mean(img)
        else:
            avg_brightness = np.mean(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY))

    # Brightness adjustment proportional to darkness
        brightness_shift = int(0.3 * (128 - avg_brightness))

    # Contrast adjustment: slightly increase for dark images
        contrast_factor = 1.0 + (0.001 * (128 - avg_brightness))  # subtle increase

        adjusted = cv2.convertScaleAbs(img, alpha=contrast_factor, beta=brightness_shift)

        print(f"{filename}: Brightened (beta={brightness_shift}, alpha={contrast_factor:.2f})")

        name, ext = os.path.splitext(filename)
        output_path = os.path.join(input_dir, f"brightened_{name}{ext}")
        cv2.imwrite(output_path, adjusted)
        print(f"✅ Saved brightened image: {output_path}")

else:
    print("Current working directory:", os.getcwd())
    print("Make sure your images are in this folder!\n")

    image_files = [
         ("dark1.jpg", 0),
         ("dark2.jpg", 0),
         ("bright1.jpg", 1),
         ("bright2.jpg", 1)
    ]

    X = []
    y = []

    for img_path, label in image_files:
        if not os.path.exists(img_path):
            print(f"❌ File NOT found: {img_path}")
            continue

        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            print(f"❌ Failed to load: {img_path}")
            continue

        avg_brightness = np.mean(img)
        X.append([avg_brightness])
        y.append(label)
        print(f"✅ Loaded {img_path}, avg brightness: {avg_brightness}")

# Convert to numpy
    X = np.array(X)
    y = np.array(y)

# Safety check
    if len(X) < 2:
        raise ValueError("❌ Not enough training images loaded.")

# Train-test split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.25, random_state=42
    )

# Train model
    model = LogisticRegression()
    model.fit(X_train, y_train)

# Evaluate
    predictions = model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    print(f"\nModel Accuracy: {accuracy:.2f}")

# -----------------------------
# TEST IMAGE PROCESSING
# -----------------------------
    test_image_path = "bright1.jpg"

    if not os.path.exists(test_image_path):
        raise FileNotFoundError(f"❌ Test image not found: {test_image_path}")

    test_image = cv2.imread(test_image_path, cv2.IMREAD_GRAYSCALE)
    if test_image is None:
        raise ValueError("❌ Failed to load test image")

    brightness = np.mean(test_image)
    prediction = model.predict([[brightness]])

# Create output folder if it doesn't exist
    output_dir = "output"
    os.makedirs(output_dir, exist_ok=True)

# Adjust brightness based on prediction
    if prediction[0] == 1:
    # Bright image → reduce brightness
        adjusted = cv2.convertScaleAbs(test_image, alpha=0.7, beta=-30)
        print("Bright image detected → reducing brightness")
    else:
    # Dark image → increase brightness
        adjusted = cv2.convertScaleAbs(test_image, alpha=1.3, beta=40)
        print("Dark image detected → increasing brightness")

# Save output
    output_path = os.path.join(output_dir, "output_adjusted.jpg")
    cv2.imwrite(output_path, adjusted)

    print(f"✅ Modified image saved in: {output_path}")

Current working directory: C:\Users\RRIYA\PycharmProjects\PythonProject\.ipynb_checkpoints\full
Make sure your images are in this folder!

✅ Loaded dark1.jpg, avg brightness: 82.55070215220591
✅ Loaded dark2.jpg, avg brightness: 82.55070215220591
✅ Loaded bright1.jpg, avg brightness: 82.55070215220591
✅ Loaded bright2.jpg, avg brightness: 82.55070215220591

Model Accuracy: 0.00
Bright image detected → reducing brightness
✅ Modified image saved in: output\output_adjusted.jpg
